# Defining DataLoader

In [71]:
from skimage import io
img = io.imread('/kaggle/input/marida-m/patches/S2_1-12-19_48MYU/S2_1-12-19_48MYU_0.tif')
mask = io.imread('/kaggle/input/marida-m/patches/S2_1-12-19_48MYU/S2_1-12-19_48MYU_0_cl.tif')
print(img.shape)
print(mask.shape)
print(img.dtype)
mask = mask.astype('int64')
print(mask.dtype)


(256, 256, 11)
(256, 256)
float32
int64


In [72]:
# -*- coding: utf-8 -*-


import os
import torch
import random
import numpy as np
from tqdm import tqdm
#from osgeo import gdal
from skimage import io
from os.path import dirname as up
from torch.utils.data import Dataset
import torchvision.transforms.functional as F

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

# Pixel-Level class distribution (total sum equals 1.0)
class_distr = torch.Tensor([0.00452, 0.00203, 0.00254, 0.00168, 0.00766, 0.15206, 0.20232,
 0.35941, 0.00109, 0.20218, 0.03226, 0.00693, 0.01322, 0.01158, 0.00052])

bands_mean = np.array([0.05197577, 0.04783991, 0.04056812, 0.03163572, 0.02972606, 0.03457443,
 0.03875053, 0.03436435, 0.0392113,  0.02358126, 0.01588816]).astype('float32')

bands_std = np.array([0.04725893, 0.04743808, 0.04699043, 0.04967381, 0.04946782, 0.06458357,
 0.07594915, 0.07120246, 0.08251058, 0.05111466, 0.03524419]).astype('float32')

###############################################################
# Pixel-level Semantic Segmentation Data Loader               #
###############################################################
#dataset_path = os.path.join(up(up(up(__file__))), 'data')
dataset_path = '/kaggle/input/marida-m/'

class GenDEBRIS(Dataset): # Extend PyTorch's Dataset class
    def __init__(self, mode = 'train', transform=None, standardization=None, path = dataset_path, agg_to_water= True):
        
        if mode=='train':
            self.ROIs = np.genfromtxt(os.path.join(path, 'splits', 'train_X.txt'),dtype='str')
                
        elif mode=='test':
            self.ROIs = np.genfromtxt(os.path.join(path, 'splits', 'test_X.txt'),dtype='str')
                
        elif mode=='val':
            self.ROIs = np.genfromtxt(os.path.join(path, 'splits', 'val_X.txt'),dtype='str')
            
        else:
            raise
            
        self.X = []           # Loaded Images
        self.y = []           # Loaded Output masks
            
        for roi in tqdm(self.ROIs, desc = 'Load '+mode+' set to memory'):
            
            # Construct file and folder name from roi
            roi_folder = '_'.join(['S2'] + roi.split('_')[:-1])               # Get Folder Name
            roi_name = '_'.join(['S2'] + roi.split('_'))                      # Get File Name
            roi_file = os.path.join(path, 'patches', roi_folder,roi_name + '.tif')       # Get File path
            roi_file_cl = os.path.join(path, 'patches', roi_folder,roi_name + '_cl.tif') # Get Class Mask
            
            # Load Classsification Mask
            #ds = gdal.Open(roi_file_cl)
            #temp = np.copy(ds.ReadAsArray().astype(np.int64))
            ds = io.imread(roi_file_cl)
            temp = np.copy(ds.astype(np.int64))
            # Aggregation
            if agg_to_water:
                temp[temp==15]=7          # Mixed Water to Marine Water Class
                temp[temp==14]=7          # Wakes to Marine Water Class
                temp[temp==13]=7          # Cloud Shadows to Marine Water Class
                temp[temp==12]=7          # Waves to Marine Water Class
            
            # Categories from 1 to 0
            temp = np.copy(temp - 1)
            ds=None                   # Close file
            
            self.y.append(temp)
            
            # Load Patch
            #ds = gdal.Open(roi_file)
            #temp = np.copy(ds.ReadAsArray())
            ds = io.imread(roi_file)
            #ds = np.moveaxis(ds,[0,1,2],[2,0,1])
            temp = np.copy(ds)
            ds=None
            self.X.append(temp)          

        self.impute_nan = np.tile(bands_mean, (temp.shape[0],temp.shape[1],1))
        self.mode = mode
        self.transform = transform
        self.standardization = standardization
        self.length = len(self.y)
        self.path = path
        self.agg_to_water = agg_to_water
        
    def __len__(self):

        return self.length
    
    def getnames(self):
        return self.ROIs
    
    def __getitem__(self, index):
        
        img = self.X[index]
        target = self.y[index]
        
        #img = np.moveaxis(img, [0, 1, 2], [2, 0, 1]).astype('float32')       # CxWxH to WxHxC
        
        #self.impute_nan = np.moveaxis(self.impute_nan, [0, 1, 2], [2, 0, 1]).astype('float32')       # CxWxH to WxHxC
        
        nan_mask = np.isnan(img)
        img[nan_mask] = self.impute_nan[nan_mask]
        
        if self.transform is not None:
            target = target[:,:,np.newaxis]
            stack = np.concatenate([img, target], axis=-1).astype('float32') # In order to rotate-transform both mask and image
        
            stack = self.transform(stack)

            img = stack[:-1,:,:]
            target = stack[-1,:,:].long()                                    # Recast target values back to int64 or torch long dtype
        
        if self.standardization is not None:
            img = self.standardization(img)
            
        return img, target
    
###############################################################
# Transformations                                             #
###############################################################
class RandomRotationTransform:
    """Rotate by one of the given angles."""

    def __init__(self, angles):
        self.angles = angles

    def __call__(self, x):
        angle = random.choice(self.angles)
        return F.rotate(x, angle)
    
###############################################################
# Weighting Function for Semantic Segmentation                #
###############################################################
def gen_weights(class_distribution, c = 1.02):
    return 1/torch.log(c + class_distribution)

# Defining U-net ++ (not working)

In [73]:
import torch
from torch import nn

class NestedUNet(nn.Module):
    def __init__(self, input_bands=11, output_classes=11, hidden_channels=16):
        super(NestedUNet, self).__init__()
        self.hidden_channels = hidden_channels

        def double_conv(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            )

        # Contracting path
        self.conv0_0 = double_conv(input_bands, hidden_channels)
        self.conv1_0 = double_conv(hidden_channels, hidden_channels * 2)
        self.conv2_0 = double_conv(hidden_channels * 2, hidden_channels * 4)
        self.conv3_0 = double_conv(hidden_channels * 4, hidden_channels * 8)
        self.conv4_0 = double_conv(hidden_channels * 8, hidden_channels * 8)

        # Expanding path (nested convolutions)
        self.conv0_1 = double_conv(hidden_channels + hidden_channels * 2, hidden_channels)
        self.conv1_1 = double_conv(hidden_channels * 2 + hidden_channels * 4, hidden_channels * 2)
        self.conv2_1 = double_conv(hidden_channels * 4 + hidden_channels * 8, hidden_channels * 4)
        self.conv3_1 = double_conv(hidden_channels * 8 + hidden_channels * 8, hidden_channels * 8)

        self.conv0_2 = double_conv(hidden_channels * 2 + hidden_channels, hidden_channels)
        self.conv1_2 = double_conv(hidden_channels * 4 + hidden_channels * 2, hidden_channels * 2)
        self.conv2_2 = double_conv(hidden_channels * 8 + hidden_channels * 4, hidden_channels * 4)

        self.conv0_3 = double_conv(hidden_channels * 3 + hidden_channels, hidden_channels)
        self.conv1_3 = double_conv(hidden_channels * 6 + hidden_channels * 2, hidden_channels * 2)

        self.conv0_4 = double_conv(hidden_channels * 4 + hidden_channels, hidden_channels)

        # Up-sampling layers
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        # Final output layer
        self.final = nn.Conv2d(hidden_channels, output_classes, kernel_size=1)

    def forward(self, x):
        # Contracting path
        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(self.up(x0_0))
        x2_0 = self.conv2_0(self.up(x1_0))
        x3_0 = self.conv3_0(self.up(x2_0))
        x4_0 = self.conv4_0(self.up(x3_0))

        # Nested path
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], dim=1))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], dim=1))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], dim=1))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], dim=1))

        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], dim=1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], dim=1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], dim=1))

        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], dim=1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], dim=1))

        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], dim=1))

        # Final output layer
        output = self.final(x0_4)
        return output


# Defining Residual Unet

In [74]:
# -*- coding: utf-8 -*-

import torch
from torch import nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels))

        self.residual = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        residual = self.residual(x)
        x = self.conv(x)
        return nn.ReLU(inplace=True)(x + residual)

class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Down, self).__init__()
        self.maxpool = nn.MaxPool2d(2)
        self.res_block = ResidualBlock(in_channels, out_channels)

    def forward(self, x):
        x = self.maxpool(x)
        return self.res_block(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Up, self).__init__()
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.res_block = ResidualBlock(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x2, x1], dim=1)
        return self.res_block(x)

class ResidualUNet(nn.Module):
    def __init__(self, input_bands=11, output_classes=11, hidden_channels=16):
        super(ResidualUNet, self).__init__()

        # Initial Convolution Layer
        self.inc = ResidualBlock(input_bands, hidden_channels)

        # Contracting Path
        self.down1 = Down(hidden_channels, 2 * hidden_channels)
        self.down2 = Down(2 * hidden_channels, 4 * hidden_channels)
        self.down3 = Down(4 * hidden_channels, 8 * hidden_channels)
        self.down4 = Down(8 * hidden_channels, 8 * hidden_channels)

        # Expanding Path
        self.up1 = Up(16 * hidden_channels, 4 * hidden_channels)
        self.up2 = Up(8 * hidden_channels, 2 * hidden_channels)
        self.up3 = Up(4 * hidden_channels, hidden_channels)
        self.up4 = Up(2 * hidden_channels, hidden_channels)

        # Output Convolution Layer
        self.outc = nn.Conv2d(hidden_channels, output_classes, kernel_size=1)

    def forward(self, x):
        # Contracting Path
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        # Expanding Path
        x6 = self.up1(x5, x4)
        x7 = self.up2(x6, x3)
        x8 = self.up3(x7, x2)
        x9 = self.up4(x8, x1)

        # Output Convolution Layer
        logits = self.outc(x9)
        return logits

# Defining Attention U-Net

In [75]:
import torch
from torch import nn

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionGate, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(F_int)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    def forward(self, g, x):
        # Align spatial dimensions if necessary
        if g.size() != x.size():
            g = self.upsample(g)

        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi


class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Down, self).__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Up, self).__init__()
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class AttentionUNet(nn.Module):
    def __init__(self, input_bands=1, output_classes=1, hidden_channels=64):
        super(AttentionUNet, self).__init__()
        # Initial Convolution Layer
        self.inc = nn.Sequential(
            nn.Conv2d(input_bands, hidden_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU(inplace=True)
        )

        # Contracting Path
        self.down1 = Down(hidden_channels, hidden_channels * 2)
        self.down2 = Down(hidden_channels * 2, hidden_channels * 4)
        self.down3 = Down(hidden_channels * 4, hidden_channels * 8)
        self.down4 = Down(hidden_channels * 8, hidden_channels * 8)

        # Expanding Path
        self.up1 = Up(hidden_channels * 16, hidden_channels * 4)
        self.up2 = Up(hidden_channels * 8, hidden_channels * 2)
        self.up3 = Up(hidden_channels * 4, hidden_channels)
        self.up4 = Up(hidden_channels * 2, hidden_channels)

        # Attention Gates
        self.att1 = AttentionGate(hidden_channels * 8, hidden_channels * 8, hidden_channels * 4)
        self.att2 = AttentionGate(hidden_channels * 4, hidden_channels * 4, hidden_channels * 2)
        self.att3 = AttentionGate(hidden_channels * 2, hidden_channels * 2, hidden_channels)

        # Output Convolution Layer
        self.outc = nn.Conv2d(hidden_channels, output_classes, kernel_size=1)

    def forward(self, x):
        # Contracting Path
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        # Expanding Path
        x6 = self.up1(x5, self.att1(x5, x4))
        x7 = self.up2(x6, self.att2(x6, x3))
        x8 = self.up3(x7, self.att3(x7, x2))
        x9 = self.up4(x8, x1)

        # Output Layer
        logits = self.outc(x9)
        return logits

# Defining SegNet (not working)

In [76]:
import torch
from torch import nn

class SegNet(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(SegNet, self).__init__()
        
        # Encoder (Downsampling)
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2)
        )
        
        # Decoder (Upsampling)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(256, 256, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(128, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(64, output_channels, kernel_size=3, stride=2, padding=1, output_padding=1)
        )
    
    def forward(self, x):
        # Forward pass through encoder
        x = self.encoder(x)
        
        # Forward pass through decoder
        x = self.decoder(x)
        
        return x


# Defining U-Net

In [77]:
# -*- coding: utf-8 -*-

import torch
import numpy as np
from torch import nn
import random

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

class Down(nn.Module):
    # Contracting Layer
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True))

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    # Expanding Layer
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True))

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class UNet(nn.Module):
    
    def __init__(self, input_bands = 11, output_classes = 11, hidden_channels=16):
        super(UNet, self).__init__()
        
        # Initial Convolution Layer
        self.inc = nn.Sequential(
            nn.Conv2d(input_bands, hidden_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU(inplace=True))
        
        # Contracting Path
        self.down1 = Down(hidden_channels, 2*hidden_channels)
        self.down2 = Down(2*hidden_channels, 4*hidden_channels)
        self.down3 = Down(4*hidden_channels, 8*hidden_channels)
        self.down4 = Down(8*hidden_channels, 8*hidden_channels)
        
        # Expanding Path
        self.up1 = Up(16*hidden_channels, 4*hidden_channels)
        self.up2 = Up(8*hidden_channels, 2*hidden_channels)
        self.up3 = Up(4*hidden_channels, hidden_channels)
        self.up4 = Up(2*hidden_channels, hidden_channels)
        
        # Output Convolution Layer
        self.outc = nn.Conv2d(hidden_channels, output_classes, kernel_size=1)

    def forward(self, x):
        # Initial Convolution Layer
        x1 = self.inc(x)
        
        # Contracting Path
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        
        # Expanding Path
        x6 = self.up1(x5, x4)
        x7 = self.up2(x6, x3)
        x8 = self.up3(x7, x2)
        x9 = self.up4(x8, x1)
        
        # Output Convolution Layer
        logits = self.outc(x9)
        return logits

# UTILS

In [78]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, jaccard_score, hamming_loss, label_ranking_loss, coverage_error
import sklearn.metrics as metr
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Evaluation for Pixel-level semantic segmentation
def Evaluation(y_predicted, y_true):

    micro_prec = precision_score(y_true, y_predicted, average='micro')
    macro_prec = precision_score(y_true, y_predicted, average='macro')
    weight_prec = precision_score(y_true, y_predicted, average='weighted')
    
    micro_rec = recall_score(y_true, y_predicted, average='micro')
    macro_rec = recall_score(y_true, y_predicted, average='macro')
    weight_rec = recall_score(y_true, y_predicted, average='weighted')
        
    macro_f1 = f1_score(y_true, y_predicted, average="macro")
    micro_f1 = f1_score(y_true, y_predicted, average="micro")
    weight_f1 = f1_score(y_true, y_predicted, average="weighted")
        
    subset_acc = accuracy_score(y_true, y_predicted)
    
    iou_acc = jaccard_score(y_true, y_predicted, average='macro')

    info = {
            "macroPrec" : macro_prec,
            "microPrec" : micro_prec,
            "weightPrec" : weight_prec,
            "macroRec" : macro_rec,
            "microRec" : micro_rec,
            "weightRec" : weight_rec,
            "macroF1" : macro_f1,
            "microF1" : micro_f1,
            "weightF1" : weight_f1,
            "subsetAcc" : subset_acc,
            "IoU": iou_acc
            }
    
    return info

# Evaluation for Multi-Label classification
def Evaluation_ML(y_predicted, predicted_probs, y_true):

    micro_prec = precision_score(y_true, y_predicted, average='micro')
    macro_prec = precision_score(y_true, y_predicted, average='macro')
    sample_prec = precision_score(y_true, y_predicted, average='samples')
    
    micro_rec = recall_score(y_true, y_predicted, average='micro')
    macro_rec = recall_score(y_true, y_predicted, average='macro')
    sample_rec = recall_score(y_true, y_predicted, average='samples')
        
    macro_f1 = f1_score(y_true, y_predicted, average="macro")
    micro_f1 = f1_score(y_true, y_predicted, average="micro")
    sample_f1 = f1_score(y_true, y_predicted, average="samples")
        
    subset_acc = accuracy_score(y_true, y_predicted)

    hamming = hamming_loss(y_true, y_predicted)
    coverage = coverage_error(y_true, y_predicted)
    rank_loss = label_ranking_loss(y_true, y_predicted)

    info = {
            "macroPrec" : macro_prec,
            "microPrec" : micro_prec,
            "samplePrec" : sample_prec,
            "macroRec" : macro_rec,
            "microRec" : micro_rec,
            "sampleRec" : sample_rec,
            "macroF1" : macro_f1,
            "microF1" : micro_f1,
            "sampleF1" : sample_f1,
            "HammingLoss" : hamming,
            "subsetAcc" : subset_acc,
            "coverageError" : coverage,
            "rankLoss" : rank_loss
            }
    return info

def confusion_matrix(y_gt, y_pred, labels):

    # compute metrics
    cm      = metr.confusion_matrix  (y_gt, y_pred)
    f1_macro= metr.f1_score          (y_gt, y_pred, average='macro')
    mPA      = metr.recall_score      (y_gt, y_pred, average='macro')
    OA      = metr.accuracy_score    (y_gt, y_pred)
    UA      = metr.precision_score   (y_gt, y_pred, average=None)
    PA      = metr.recall_score      (y_gt, y_pred, average=None)
    f1      = metr.f1_score          (y_gt, y_pred, average=None)
    IoC     = metr.jaccard_score     (y_gt, y_pred, average=None)
    mIoC     = metr.jaccard_score    (y_gt, y_pred, average='macro')
      
    # confusion matrix
    sz1, sz2 = cm.shape
    cm_with_stats             = np.zeros((sz1+4,sz2+2))
    cm_with_stats[0:-4, 0:-2] = cm
    cm_with_stats[-3  , 0:-2] = np.round(IoC,2)
    cm_with_stats[-2  , 0:-2] = np.round(UA,2)
    cm_with_stats[-1  , 0:-2] = np.round(f1,2)
    cm_with_stats[0:-4,   -1] = np.round(PA,2)
    
    cm_with_stats[-4  , 0:-2] = np.sum(cm, axis=0) 
    cm_with_stats[0:-4,   -2] = np.sum(cm, axis=1)
    
    # convert to list
    cm_list = cm_with_stats.tolist()
    
    # first row
    first_row = []
    first_row.extend (labels)
    first_row.append ('Sum')
    first_row.append ('Recall')
    
    # first col
    first_col = []
    first_col.extend(labels)
    first_col.append ('Sum')
    first_col.append ('IoU')
    first_col.append ('Precision')
    first_col.append ('F1-score')
    
    # fill rest of the text 
    idx = 0
    for sublist in cm_list:
        if   idx == sz1:
            sublist[-2]  = 'mPA:'
            sublist[-1]  = round(mPA,2)
            cm_list[idx] = sublist
        elif   idx == sz1+1:
            sublist[-2]  = 'mIoU:'
            sublist[-1]  = round(mIoC,2)
            cm_list[idx] = sublist
            
        elif idx == sz1+2:
            sublist[-2]  = 'OA:'
            sublist[-1]  = round(OA,2)
            cm_list[idx] = sublist
            
        elif idx == sz1+3:
            cm_list[idx] = sublist
            sublist[-2]  = 'F1-macro:'
            sublist[-1]  = round(f1_macro,2)    
        idx +=1
    
    # Convert to data frame
    df = pd.DataFrame(np.array(cm_list))
    df.columns = first_row
    df.index = first_col
    
    return df

def print_confusion_matrix_ML(confusion_matrix, class_label, ind_names, col_names):

    df_cm = pd.DataFrame(confusion_matrix, index=ind_names, columns=col_names)
    
    df_cm.index.name = class_label
    return df_cm


# Training

## Settings: Training settings. Here you have to set epochs, batch_size...

In [79]:
import os
import ast
import sys
import json
import random
import logging
import argparse
import numpy as np
from tqdm import tqdm
from os.path import dirname as up
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

#sys.path.append(up(os.path.abspath(__file__)))
#from unet import UNet
#from dataloader import GenDEBRIS, bands_mean, bands_std, RandomRotationTransform , class_distr, gen_weights

#sys.path.append(os.path.join(up(up(up(os.path.abspath(__file__)))), 'utils'))
#from metrics import Evaluation

root_path = '/kaggle/working/'#up(up(up(os.path.abspath(__file__))))

logging.basicConfig(filename=os.path.join(root_path, 'logs','log_unet.log'), filemode='a',level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')
logging.info('*'*10)

def seed_all(seed):
    # Pytorch Reproducibility
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
def seed_worker(worker_id):
    # DataLoader Workers Reproducibility
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

    
import ast 
options = {}
options['agg_to_water'] = True #default=True, type=bool,  help='Aggregate Mixed Water, Wakes, Cloud Shadows, Waves with Marine Water'
options['mode']='train' # help='select between train or test ')
options['epochs']=45    # type=int, help='Number of epochs to run')
options['batch']=5      # default=5, type=int, help='Batch size')
options['resume_from_epoch']=0  # default=0, type=int, help='load model from previous epoch')

options['input_channels']=11 # default=11, type=int, help='Number of input bands')
options['output_channels']=11 # default=11, type=int, help='Number of output classes')
options['hidden_channels']=16 # default=16, type=int, help='Number of hidden features')
options['weight_param']= 1.03 # default=1.03, type=float, help='Weighting parameter for Loss Function')

# Optimization
options['lr'] = 2e-4 # default=2e-4, type=float, help='learning rate')
options['decay']=0 # default=0, type=float, help='learning rate decay')
options['reduce_lr_on_plateau']=0 # default=0, type=int, help='reduce learning rate when no increase (0 or 1)')
options['lr_steps'] = '40' # default='[40]', type=str, help='Specify the steps that the lr will be reduced')

# Evaluation/Checkpointing
options['checkpoint_path']='/kaggle/working/trained_models/' #default='/kaggle/working/trained_models/',to save checkpoints into (empty = this folder)')
options['eval_every']=1 # default=1, type=int, help='How frequently to run evaluation (epochs)')

# misc
#parser.add_argument('--num_workers', default=1, type=int, help='How many cpus for loading data (0 is the main process)')
#parser.add_argument('--pin_memory', default=False, type=bool, help='Use pinned memory or not')
#parser.add_nextargument('--prefetch_factor', default=1, type=int, help='Number of sample loaded in advance by each worker')
#parser.add_argument('--persistent_workers', default=True, type=bool, help='This allows to maintain the workers Dataset instances alive.')
#parser.add_argument('--tensorboard', default='tsboard_segm', type=str, help='Name for tensorboard run')
options['num_workers']=1
options['pin_memory']=False
options['prefetch_factor']=1
options['persistent_workers']=True
options['tensorboard']='tsboard_segm'
#args = parser.parse_args()
#options = vars(args)  # convert to ordinary dict

# lr_steps list or single float
lr_steps = ast.literal_eval(options['lr_steps'])
#lr_steps = options['lr_steps']
if type(lr_steps) is list:
    pass
elif type(lr_steps) is int:
    lr_steps = [lr_steps]
else:
    raise
    
options['lr_steps'] = lr_steps

logging.info('parsed input parameters:')
logging.info(json.dumps(options, indent = 2))

## Training Section

In [80]:
import torchvision.transforms.functional as F

seed_all(0)
g = torch.Generator()
g.manual_seed(0)

# Tensorboard
writer = SummaryWriter(os.path.join(root_path, 'logs', options['tensorboard']))

# Transformations
transform_train = transforms.Compose([transforms.ToTensor(),
                                RandomRotationTransform([-90, 0, 90, 180]),
                                transforms.RandomHorizontalFlip()])

transform_test = transforms.Compose([transforms.ToTensor()])

standardization = transforms.Normalize(bands_mean, bands_std)

# Construct Data loader
dataset_train = GenDEBRIS('train', transform=transform_train, standardization=standardization, agg_to_water=options['agg_to_water'])
dataset_test = GenDEBRIS('val', transform=transform_test, standardization=standardization, agg_to_water=options['agg_to_water'])

train_loader = DataLoader(dataset_train,
                          batch_size=options['batch'],
                          shuffle=True,
                          num_workers=options['num_workers'],
                          pin_memory=options['pin_memory'],
                          prefetch_factor=options['prefetch_factor'],
                          persistent_workers=options['persistent_workers'],
                          worker_init_fn=seed_worker,
                          generator=g)

test_loader = DataLoader(dataset_test,
                         batch_size=options['batch'],
                         shuffle=False,
                         num_workers=options['num_workers'],
                         pin_memory=options['pin_memory'],
                         prefetch_factor=options['prefetch_factor'],
                         persistent_workers=options['persistent_workers'],
                         worker_init_fn=seed_worker,
                         generator=g)

# Use gpu or cpu
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


# # SegNet
# model = SegNet(input_channels=options['input_channels'], output_channels=options['output_channels'])


# Residual U-Net
# model =  ResidualUNet(input_bands=options['input_channels'],
#              output_classes=options['output_channels'],
#              hidden_channels=options['hidden_channels'])


# Attention U-Net
# model = AttentionUNet(input_bands=options['input_channels'],
#              output_classes=options['output_channels'],
#              hidden_channels=options['hidden_channels'])

# U-Net
model = UNet(input_bands=options['input_channels'],
             output_classes=options['output_channels'],
             hidden_channels=options['hidden_channels'])

# U-Net++

# model = NestedUNet(input_bands=options['input_channels'],
#              output_classes=options['output_channels'],
#              hidden_channels=options['hidden_channels'])


# FCN
# model = FCN(input_channels=options['input_channels'], output_classes=options['output_channels'])

model.to(device)

# Load model from specific epoch to continue the training or start the evaluation
if options['resume_from_epoch'] > 1:
    resume_model_dir = os.path.join(options['checkpoint_path'], str(options['resume_from_epoch']))
    model_file = os.path.join(resume_model_dir, 'model.pth')
    logging.info('Loading model files from folder: %s' % model_file)

    checkpoint = torch.load(model_file, map_location=device)
    model.load_state_dict(checkpoint)

    del checkpoint  # dereference
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

global class_distr
# Aggregate Distribution Mixed Water, Wakes, Cloud Shadows, Waves with Marine Water
if options['agg_to_water']:
    agg_distr = sum(class_distr[-4:])  # Density of Mixed Water, Wakes, Cloud Shadows, Waves
    class_distr[6] += agg_distr       # To Water
    class_distr = class_distr[:-4]    # Drop Mixed Water, Wakes, Cloud Shadows, Waves

# Weighted Cross Entropy Loss & adam optimizer
weight = gen_weights(class_distr, c=options['weight_param'])
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='mean', weight=weight.to(device))

optimizer = torch.optim.Adam(model.parameters(), lr=options['lr'], weight_decay=options['decay'])

# Learning Rate scheduler
if options['reduce_lr_on_plateau'] == 1:
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)
else:
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, options['lr_steps'], gamma=0.1, verbose=True)

# Start training
start = options['resume_from_epoch'] + 1
epochs = options['epochs']
eval_every = options['eval_every']

# Write model-graph to Tensorboard
t_loss = []
v_loss = []
v_acc = []

dataiter = iter(train_loader)
image_temp, _ = next(dataiter)
# writer.add_graph(model, image_temp.to(device))

###############################################################
# Start Training                                              #
###############################################################
model.train()

for epoch in range(start, epochs + 1):
    training_loss = []
    training_batches = 0

    i_board = 0
    for (image, target) in tqdm(train_loader, desc="training"):
        image = image.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        logits = model(image)

        loss = criterion(logits, target)

        loss.backward()

        training_batches += target.shape[0]

        training_loss.append((loss.data * target.shape[0]).tolist())

        optimizer.step()

        # Write running loss
        writer.add_scalar('training loss', loss, (epoch - 1) * len(train_loader) + i_board)
        i_board += 1

    t_loss.append(sum(training_loss) / training_batches)
    logging.info("Training loss was: " + str(sum(training_loss) / training_batches))
    print(f"Epoch {epoch}: Training loss was: {sum(training_loss) / training_batches}")

    ###############################################################
    # Start Evaluation                                            #
    ###############################################################

    if epoch % eval_every == 0 or epoch == 1:
        model.eval()

        test_loss = []
        test_batches = 0
        y_true = []
        y_predicted = []

        with torch.no_grad():
            for (image, target) in tqdm(test_loader, desc="testing"):
                image = image.to(device)
                target = target.to(device)

                logits = model(image)

                loss = criterion(logits, target)

                # Accuracy metrics only on annotated pixels
                logits = torch.movedim(logits, (0, 1, 2, 3), (0, 3, 1, 2))
                logits = logits.reshape((-1, options['output_channels']))
                target = target.reshape(-1)
                mask = target != -1
                logits = logits[mask]
                target = target[mask]

                probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
                target = target.cpu().numpy()

                test_batches += target.shape[0]
                test_loss.append((loss.data * target.shape[0]).tolist())
                y_predicted += probs.argmax(1).tolist()
                y_true += target.tolist()

            y_predicted = np.asarray(y_predicted)
            y_true = np.asarray(y_true)

            ####################################################################
            # Save Scores to the .log file and visualize also with tensorboard #
            ####################################################################
            acc = Evaluation(y_predicted, y_true)
            logging.info("\n")
            logging.info("Test loss was: " + str(sum(test_loss) / test_batches))
            logging.info("STATISTICS AFTER EPOCH " + str(epoch) + ": \n")
            logging.info("Evaluation: " + str(acc))
            print(f"Epoch {epoch}: Test loss was: {sum(test_loss) / test_batches}")
            print(f"STATISTICS AFTER EPOCH {epoch}: \n")
            print(f"Evaluation: {acc}")
            v_loss.append(sum(test_loss) / test_batches)
            v_acc.append(acc['subsetAcc'])

            logging.info("Saving models")
            model_dir = os.path.join(options['checkpoint_path'], str(epoch))
            model_dir = options['checkpoint_path'] + str(epoch)
            os.makedirs(model_dir, exist_ok=True)
            torch.save(model.state_dict(), os.path.join(model_dir, 'model.pth'))

            writer.add_scalars('Loss per epoch', {'Test loss': sum(test_loss) / test_batches,
                                                  'Train loss': sum(training_loss) / training_batches},
                               epoch)

            writer.add_scalar('Precision/test macroPrec', acc["macroPrec"], epoch)
            writer.add_scalar('Precision/test microPrec', acc["microPrec"], epoch)
            writer.add_scalar('Precision/test weightPrec', acc["weightPrec"], epoch)

            writer.add_scalar('Recall/test macroRec', acc["macroRec"], epoch)
            writer.add_scalar('Recall/test microRec', acc["microRec"], epoch)
            writer.add_scalar('Recall/test weightRec', acc["weightRec"], epoch)

            writer.add_scalar('F1/test macroF1', acc["macroF1"], epoch)
            writer.add_scalar('F1/test microF1', acc["microF1"], epoch)
            writer.add_scalar('F1/test weightF1', acc["weightF1"], epoch)

            writer.add_scalar('IoU/test MacroIoU', acc["IoU"], epoch)

        if options['reduce_lr_on_plateau'] == 1:
            scheduler.step(sum(test_loss) / test_batches)
        else:
            scheduler.step()

        model.train()

# plt.figure()
# plt.plot(t_loss, 'r')
# plt.plot(v_loss, 'b')
# plt.legend(['train', 'validation'])
# plt.figure()
# plt.plot(v_acc, 'r')
# Plotting the losses over epochs
plt.figure()
plt.plot(t_loss, 'r', label='Training Loss')
plt.plot(v_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss_plot.png')  # Optional: Save the plot
plt.show()

# Plotting the validation accuracy over epochs
plt.figure()
plt.plot(v_acc, 'r', label='Validation Accuracy')
plt.title('Validation Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accuracy_plot.png')  # Optional: Save the plot
plt.show()

training:   6%|▌         | 8/139 [00:10<02:54,  1.33s/it]


KeyboardInterrupt: 

# Testing

In [ ]:
dataset_test = GenDEBRIS('test', transform=transform_test, standardization=standardization, agg_to_water=options['agg_to_water'])

test_loader = DataLoader(dataset_test,
                        batch_size=1,
                        shuffle=False,
                        num_workers=options['num_workers'],
                        pin_memory=options['pin_memory'],
                        prefetch_factor=options['prefetch_factor'],
                        persistent_workers=options['persistent_workers'],
                        worker_init_fn=seed_worker,
                        generator=g)

model.eval()

test_loss = []
test_batches = 0
y_true = []
y_predicted = []

i = 0
with torch.no_grad():
    for (image, target) in tqdm(test_loader, desc="testing"):
        image = image.to(device)
        target = target.to(device)

        logits_ = model(image)
        plt.figure()
        plt.subplot(1, 3, 1), plt.imshow(np.squeeze(image.permute(0, 2, 3, 1).detach().cpu()[:, :, :, [3, 2, 1]]))
        plt.subplot(1, 3, 2), plt.imshow(np.squeeze(torch.argmax(logits_, 1).detach().cpu()), cmap='viridis', clim=[0, 11])
        plt.subplot(1, 3, 3), plt.imshow(np.squeeze(target.detach().cpu()), cmap='viridis', clim=[0, 11])

        loss = criterion(logits_, target)

        # Accuracy metrics only on annotated pixels
        logits = torch.movedim(logits_, (0, 1, 2, 3), (0, 3, 1, 2))
        logits = logits.reshape((-1, options['output_channels']))
        target = target.reshape(-1)
        mask = target != -1
        logits = logits[mask]
        target = target[mask]

        probs = torch.nn.functional.softmax(logits, dim=1).cpu().numpy()
        target = target.cpu().numpy()

        test_batches += target.shape[0]
        test_loss.append((loss.data * target.shape[0]).tolist())
        y_predicted += probs.argmax(1).tolist()
        y_true += target.tolist()

    y_predicted = np.asarray(y_predicted)
    y_true = np.asarray(y_true)

    ####################################################################
    # Save Scores to the .log file                                     #
    ####################################################################
    acc = Evaluation(y_predicted, y_true)
    logging.info("\n")
    logging.info("Test loss was: " + str(sum(test_loss) / test_batches))
    logging.info("STATISTICS: \n")
    logging.info("Evaluation: " + str(acc))

    # Print results to the console
    print(f"Test loss was: {sum(test_loss) / test_batches}")
    print("STATISTICS: \n")
    print(f"Evaluation: {acc}")
